### 参数管理
我们首先关注具有单隐藏层的多层感知机

In [2]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.0811],
        [0.1080]], grad_fn=<AddmmBackward0>)

**参数访问**

In [7]:
print(net[2].state_dict())  # state_dict()返回一个从参数名称映射到参数Tensor的字典，也是一个OrderedDict

OrderedDict([('weight', tensor([[ 0.1908, -0.3356,  0.3059, -0.0915, -0.2010, -0.0697,  0.2919,  0.2058]])), ('bias', tensor([0.0331]))])


**目标参数**

In [16]:
print(type(net[2].bias))    # Parameter是一个可以优化的参数类型
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0331], requires_grad=True)
tensor([0.0331])


In [18]:
net[2].weight.grad == None  # 还没有计算梯度

True

**一次性访问所有参数**

In [34]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])
# 1. `net[0]`表示模型中的第一个子模块。神经网络模型通常由许多层（或子模块）组成，通过索引来访问特定的子模块。这里的`net[0]`表示访问模型中的第一个子模块。
# 2. `net[0].named_parameters()`返回一个生成器，该生成器会产生子模块（在这种情况下是第一个子模块）的参数及其名称。每个参数都是一个张量，而名称是一个字符串。
# 3. `name`是参数的名称，`param`是参数的张量。
# 4. `[(name, param.shape) for name, param in net[0].named_parameters()]`是一个列表推导式，它遍历`net[0]`子模块的参数，并为每个参数生成一个元组，元组包含参数的名称和形状。
# 5. `print(*[(name, param.shape) for name, param in net[0].named_parameters()])`使用`print`函数打印了这个列表中的每个元组。`*`符号用于展开列表，使其成为`print`函数的多个参数，从而以易读的方式打印出参数名称和形状。
# 6. `net.named_parameters()`返回一个生成器，该生成器会产生整个模型的所有参数及其名称。每个参数都是一个张量，而名称是一个字符串。
# 7. 同样地，`[(name, param.shape) for name, param in net.named_parameters()]`是一个列表推导式，它遍历整个模型的参数，并为每个参数生成一个元组，元组包含参数的名称和形状。
# 8. `print(*[(name, param.shape) for name, param in net.named_parameters()])`使用`print`函数打印了整个模型的所有参数的名称和形状。
# 综上所述，这段代码用于打印模型中特定子模块（例如第一个子模块）以及整个模型的参数名称和形状，以帮助你了解模型的结构和参数情况。

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [35]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[-0.4256, -0.3368,  0.0355, -0.2054],
                      [-0.4452, -0.3629,  0.3095,  0.0042],
                      [-0.0308,  0.0642, -0.0702, -0.3943],
                      [-0.3609, -0.0623,  0.2616, -0.1050],
                      [ 0.0984,  0.4006,  0.1920, -0.0916],
                      [ 0.0178, -0.2304,  0.3083,  0.0837],
                      [ 0.0703,  0.4019, -0.2974, -0.0267],
                      [ 0.0371,  0.0272, -0.1045, -0.3323]])),
             ('0.bias',
              tensor([-0.2332, -0.4727,  0.0624,  0.4416, -0.2444,  0.3194,  0.4213,  0.0194])),
             ('2.weight',
              tensor([[ 0.1908, -0.3356,  0.3059, -0.0915, -0.2010, -0.0697,  0.2919,  0.2058]])),
             ('2.bias', tensor([0.0331]))])

In [36]:
net.state_dict()['2.bias'].data

tensor([0.0331])

**从嵌套块收集参数**

In [42]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block{i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.4297],
        [-0.4297]], grad_fn=<AddmmBackward0>)

In [43]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)
